# Face-Detection

In [1]:
!pip install azure.cognitiveservices.vision.face

In [2]:
import os
import cv2
import requests

from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials

In [3]:
SUBSCRIPTION_KEY = 'face_api_key'

FACE_LOCATION = 'eastus'

face_base_url = 'https://{}.api.cognitive.microsoft.com'.format(FACE_LOCATION)
face_client = FaceClient(face_base_url, CognitiveServicesCredentials(SUBSCRIPTION_KEY))

## Data

In [4]:
# 타겟 이미지(하나의 얼굴)
remote_image_URL_1 = "https://www.whitehouse.gov/wp-content/uploads/2021/04/P20210303AS-1901-cropped.jpg?w=1536"

# 찾고자하는 이미지(여러개의 얼굴)
remote_image_URL_2 = "https://s3.amazonaws.com/cms.ipressroom.com/401/files/202306/64884603a1383572d389f375_Biden_Harris_750x500/Biden_Harris_750x500_1fee04a2-06a2-4d51-a67b-6caf4343d977-prv.jpg"

### remote_image_URL_1

![](https://www.whitehouse.gov/wp-content/uploads/2021/04/P20210303AS-1901-cropped.jpg?w=1536)

### remote_image_URL_2

![](https://s3.amazonaws.com/cms.ipressroom.com/401/files/202306/64884603a1383572d389f375_Biden_Harris_750x500/Biden_Harris_750x500_1fee04a2-06a2-4d51-a67b-6caf4343d977-prv.jpg)

## Detect

In [6]:
def detect_faces(face_client, image_url):
	print ("Detecting faces...")
	detected_faces = face_client.face.detect_with_url(url=image_url)
	if not detected_faces:
		raise Exception('No face detected from image {}'.format(image_url))
	if not detected_faces[0]:
		raise Exception("Parameter return_face_id of detect_with_stream or detect_with_url must be set to true (by default) for recognition purpose.")
	return detected_faces

In [7]:
# @face_ID 와 @face_IDs 에서 유사한 얼굴 찾기
def find_similar_faces(face_client, face_ID, face_IDs):
	print("Finding similar faces ...")
	return face_client.face.find_similar(face_id=face_ID, face_ids=face_IDs)


In [18]:
# 첫번째 이미지에서 얼굴 탐지
faces_1 = detect_faces(face_client, remote_image_URL_1)
if not faces_1[0]:
	print("No faces detected in " + remote_image_URL_1 + ".")
else:
	print("Face IDs of faces detected in " + remote_image_URL_1 + ":")
	for x in faces_1: print (x.face_id)

	print("Using first face ID.")
	face_ID = faces_1[0].face_id

	# 두번째 이미지에서 얼굴 탐지(리스트)
	faces_2 = detect_faces(face_client, remote_image_URL_2)
	if not faces_2[0]:
		print("No faces detected in " + remote_image_URL_2 + ".")
	else:
		print("Face IDs of faces detected in " + remote_image_URL_2 + ":")
		for x in faces_2: print (x.face_id)

		# 두번째 이미지 얼굴들에서 첮번째 얼굴과 유사한 것 탐지
		similar_faces = find_similar_faces(face_client, face_ID, list(map(lambda x: x.face_id, faces_2)))
		if not similar_faces[0]:
			print("No similar faces found in " + remote_image_URL_2 + ".")
		else:
			print("Similar faces found in " + remote_image_URL_2 + ":")
			for face in similar_faces:
				face_ID = face.face_id

				face_info = next(x for x in faces_2 if x.face_id == face_ID)
				if face_info:
					print("Face ID: " + face_ID)
					print("Face rectangle:")
					print("Left: " + str(face_info.face_rectangle.left))
					print("Top: " + str(face_info.face_rectangle.top))
					print("Width: " + str(face_info.face_rectangle.width))
					print("Height: " + str(face_info.face_rectangle.height))

					for image_url, filename in zip([remote_image_URL_1, remote_image_URL_2], ['image_target.png', 'image.png']):
						images_dir = './result'
						if not os.path.exists(images_dir):
							os.makedirs(images_dir)

						with open(os.path.join(images_dir, filename), 'wb') as f:
							response = requests.get(image_url, stream=True, verify=False)
							f.write(response.content)

					image = cv2.imread('./result/image.png')

					cv2.rectangle(image, (face_info.face_rectangle.left, face_info.face_rectangle.top), (face_info.face_rectangle.left + face_info.face_rectangle.width, face_info.face_rectangle.top + face_info.face_rectangle.height), (0, 255, 0), 2)
					cv2.imwrite('./result/face_detect.png', image)

Detecting faces...
Face IDs of faces detected in https://www.whitehouse.gov/wp-content/uploads/2021/04/P20210303AS-1901-cropped.jpg?w=1536:
7fd0be11-4f7a-4ce2-8e76-181e06eb08ce
Using first face ID.
Detecting faces...
Face IDs of faces detected in https://s3.amazonaws.com/cms.ipressroom.com/401/files/202306/64884603a1383572d389f375_Biden_Harris_750x500/Biden_Harris_750x500_1fee04a2-06a2-4d51-a67b-6caf4343d977-prv.jpg:
37edf589-21aa-480a-ab75-865ae352211e
02cd27f0-4897-4610-9ac9-756a20c334e8
Finding similar faces ...
Similar faces found in https://s3.amazonaws.com/cms.ipressroom.com/401/files/202306/64884603a1383572d389f375_Biden_Harris_750x500/Biden_Harris_750x500_1fee04a2-06a2-4d51-a67b-6caf4343d977-prv.jpg:
Face ID: 37edf589-21aa-480a-ab75-865ae352211e
Face rectangle:
Left: 220
Top: 82
Width: 112
Height: 112


/Users/loa/anaconda3/envs/codezero/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.whitehouse.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/loa/anaconda3/envs/codezero/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 's3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Result
![](https://github.com/riverallzero/azure-python/assets/93754504/36a84560-aa41-483a-8baa-643f24b8a300)